In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import color

In [29]:
def detect_harris_corners(image_path, output_path):
    """
    对输入图像进行 Harris 角点检测，并在角点位置标记红色后保存结果
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_f = np.float32(gray)
    harris_response = cv2.cornerHarris(gray_f, blockSize=2, ksize=3, k=0.04)
    harris_response = cv2.dilate(harris_response, None)
    img[harris_response > 0.01 * harris_response.max()] = [0, 0, 255]
    cv2.imwrite(output_path, img)

In [30]:
def extract_sift_features(image_path):
    """
    使用 SIFT 提取图像的关键点和描述子
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return keypoints, descriptors

In [31]:
def extract_hog_features(image_path, patch_size=32):
    """
    利用 Harris 算法检测角点，然后对每个角点周围提取固定大小（patch_size x patch_size）的图像块，
    并计算 HOG 描述子，返回关键点（cv2.KeyPoint 列表）和描述子（numpy 数组）。
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gray_f = np.float32(img)
    harris_response = cv2.cornerHarris(gray_f, blockSize=2, ksize=3, k=0.04)
    harris_response = cv2.dilate(harris_response, None)
    threshold = 0.01 * harris_response.max()
    # 获取角点坐标 (y, x)
    coords = np.argwhere(harris_response > threshold)
    
    keypoints = []
    descriptors = []
    half = patch_size // 2
    for (y, x) in coords:
        # 确保图像块在图像内
        if x - half < 0 or x + half >= img.shape[1] or y - half < 0 or y + half >= img.shape[0]:
            continue
        patch = img[y-half:y+half, x-half:x+half]
        hog_desc = hog(patch, pixels_per_cell=(8,8), cells_per_block=(2,2), feature_vector=True)
        descriptors.append(hog_desc)
        # 转换 x,y 为 float 类型构造 cv2.KeyPoint 对象
        keypoints.append(cv2.KeyPoint(float(x), float(y), patch_size))
    
    if len(descriptors) == 0:
        descriptors = None
    else:
        descriptors = np.array(descriptors, dtype=np.float32)
        # 如果只有一个角点，则确保 descriptors 为二维数组
        if descriptors.ndim == 1:
            descriptors = descriptors[np.newaxis, :]
    return keypoints, descriptors



In [32]:
def match_features(desc1, desc2):
    """
    使用暴力匹配器对两组描述子进行匹配，并返回按距离排序的匹配结果
    """
    # 确保 descriptor 为连续的 numpy 数组
    desc1 = np.ascontiguousarray(desc1, dtype=np.float32)
    desc2 = np.ascontiguousarray(desc2, dtype=np.float32)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

In [33]:
def extract_and_match_features(img1_path, img2_path, method, match_output_path):
    """
    根据所选描述子类型（SIFT 或 HOG），提取图像的关键点与描述子，然后匹配，
    并将匹配结果绘制后保存到指定路径。
    """
    if method.upper() == "SIFT":
        kp1, desc1 = extract_sift_features(img1_path)
        kp2, desc2 = extract_sift_features(img2_path)
    elif method.upper() == "HOG":
        kp1, desc1 = extract_hog_features(img1_path)
        kp2, desc2 = extract_hog_features(img2_path)
    else:
        raise ValueError("Unsupported method. Please use 'SIFT' or 'HOG'.")
    
    if desc1 is None or desc2 is None or len(kp1) == 0 or len(kp2) == 0:
        raise ValueError("No descriptors found in one of the images.")
    
    matches = match_features(desc1, desc2)
    
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=2)
    cv2.imwrite(match_output_path, img_matches)

In [73]:
def stitch_images(img1_path, img2_path, method, output_path):
    """
    使用 SIFT 或 HOG 提取关键点并匹配，利用 RANSAC 计算仿射变换矩阵，将第二张图拼接到第一张图上。
    该版本支持自动扩展画布，避免错位问题。
    """
    if method.upper() == "SIFT":
        kp1, desc1 = extract_sift_features(img1_path)
        kp2, desc2 = extract_sift_features(img2_path)
    elif method.upper() == "HOG":
        kp1, desc1 = extract_hog_features(img1_path)
        kp2, desc2 = extract_hog_features(img2_path)
    else:
        raise ValueError("Unsupported method. Please use 'SIFT' or 'HOG'.")

    if desc1 is None or desc2 is None or len(kp1) == 0 or len(kp2) == 0:
        raise ValueError("No descriptors found in one of the images.")

    matches = match_features(desc1, desc2)
    if len(matches) < 3:
        raise ValueError("Not enough matches to compute transformation.")

    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    M, mask = cv2.estimateAffinePartial2D(src_pts, dst_pts, method=cv2.RANSAC)

    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    corners_img2 = np.array([
        [0, 0], [w2, 0], [w2, h2], [0, h2]
    ], dtype=np.float32).reshape(-1, 1, 2)
    transformed_corners = cv2.transform(corners_img2, M)

    # 合并所有相关角点
    img1_corners = np.array([
        [0, 0], [w1, 0], [w1, h1], [0, h1]
    ], dtype=np.float32).reshape(-1, 1, 2)
    all_corners = np.concatenate(
        (transformed_corners, img1_corners),
        axis=0
    )

    x_coords = all_corners[:, :, 0]
    y_coords = all_corners[:, :, 1]
    x_min, x_max = np.floor(np.min(x_coords)), np.ceil(np.max(x_coords))
    y_min, y_max = np.floor(np.min(y_coords)), np.ceil(np.max(y_coords))
    output_width = int(x_max - x_min)
    output_height = int(y_max - y_min)

    # 构建平移矩阵
    translation_matrix = np.array([
        [1, 0, -x_min],
        [0, 1, -y_min],
        [0, 0, 1]
    ], dtype=np.float32)

    M = np.vstack([M, [0, 0, 1]])

    transformation_matrix = translation_matrix @ M
    transformation_matrix = transformation_matrix[:2, :]

    warped_img1 = cv2.warpAffine(img1, transformation_matrix, (output_width, output_height))
    warped_img2 = cv2.warpAffine(img2, translation_matrix[:2, :], (output_width, output_height))

    # 为每个图像创建掩膜（非零像素设置为1）
    mask1 = (warped_img1.sum(axis=2) > 0).astype(np.uint8)
    mask2 = (warped_img2.sum(axis=2) > 0).astype(np.uint8)

    # 利用距离变换计算每个像素离边缘的距离
    dist1 = cv2.distanceTransform(mask1, cv2.DIST_L2, 5)
    dist2 = cv2.distanceTransform(mask2, cv2.DIST_L2, 5)

    # 防止除零错误，加入极小值 epsilon
    epsilon = 1e-5
    combined = dist1 + dist2 + epsilon
    w1 = dist1 / combined
    w2 = dist2 / combined

    # 扩展权重通道以便与图像对应
    w1 = np.expand_dims(w1, axis=2)
    w2 = np.expand_dims(w2, axis=2)

    # 转换图像为 float 类型用于加权计算
    warped_img1_f = warped_img1.astype(np.float32)

    # 初始化结果图像
    result = np.zeros_like(warped_img1_f)

    # 确定重叠区域
    overlap = np.logical_and(mask1, mask2).astype(bool)

    # 提取重叠区域的亮度通道（转为HSV空间）
    hsv1 = cv2.cvtColor(warped_img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(warped_img2, cv2.COLOR_BGR2HSV)
    v1 = hsv1[overlap, 2].astype(np.float32)
    v2 = hsv2[overlap, 2].astype(np.float32)

    # 计算亮度差异的缩放因子（避免除以零）
    mean_v1 = np.mean(v1) + 1e-5
    mean_v2 = np.mean(v2) + 1e-5
    scale_factor = mean_v1 / mean_v2

    # 对图像2的亮度通道进行校正
    hsv2[:, :, 2] = np.clip(hsv2[:, :, 2] * scale_factor, 0, 255).astype(np.uint8)
    warped_img2 = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)
    warped_img2_f = warped_img2.astype(np.float32)

    # 对权重进行高斯模糊（核大小设为15，标准差3）
    w1_blur = cv2.GaussianBlur(w1, (15, 15), 3)
    w2_blur = cv2.GaussianBlur(w2, (15, 15), 3)

    # 重新归一化权重（防止总和超过1）
    sum_weights = w1_blur + w2_blur + 1e-5
    w1_final = w1_blur / sum_weights
    w2_final = w2_blur / sum_weights
    w1_final = np.expand_dims(w1_final, axis=2)  # 形状从 (H,W) 变为 (H,W,1)
    w2_final = np.expand_dims(w2_final, axis=2)

    # 重叠区域使用模糊后的权重融合
    result[overlap] = warped_img1_f[overlap] * w1_final[overlap] + warped_img2_f[overlap] * w2_final[overlap]

    # 非重叠区域使用原图，但添加过渡（避免硬边界）
    non_overlap1 = np.logical_and(mask1, ~overlap)
    non_overlap2 = np.logical_and(mask2, ~overlap)

    # 对非重叠区域边缘进行轻微模糊
    mask1_blur = cv2.GaussianBlur(mask1.astype(np.float32), (5,5), 1)
    mask2_blur = cv2.GaussianBlur(mask2.astype(np.float32), (5,5), 1)
    result[non_overlap1] = warped_img1_f[non_overlap1] * mask1_blur[non_overlap1, None]
    result[non_overlap2] += warped_img2_f[non_overlap2] * mask2_blur[non_overlap2, None]

    # 转换回 uint8 类型并输出结果
    result = np.clip(result, 0, 255).astype(np.uint8)
    cv2.imwrite(output_path, result)

In [45]:
def stitch_multiple_images(image_paths, output_path):
    """
    基于 SIFT + RANSAC 的方法，依次对多幅图像进行拼接，
    最终将拼接结果保存到 output_path 中。
    """
    if len(image_paths) < 2:
        raise ValueError("Need at least 2 images")

    # 初始化基准图像
    base_img = cv2.imread(image_paths[0])
    cv2.imwrite(output_path, base_img)

    # 逐个拼接后续图像
    for img_path in image_paths[1:]:
        try:
            stitch_images(output_path, img_path, "SIFT", output_path)
        except Exception as e:
            print(f"Skipping {img_path} due to error: {str(e)}")
            continue

    # 保存最终结果
    final_result = cv2.imread(output_path)
    cv2.imwrite(output_path, final_result)


In [36]:
# 检测角点
detect_harris_corners("images/sudoku.png", "results/sudoku_keypoints.png")

In [37]:
# 提取 SIFT 特征
extract_and_match_features("images/uttower1.jpg", "images/uttower2.jpg", "SIFT", "results/uttower_match_sift.png")

In [38]:
# 提取 HOG 特征
extract_and_match_features("images/uttower1.jpg", "images/uttower2.jpg", "HOG", "results/uttower_match_hog.png")

In [74]:
# 拼接两张图像
stitch_images("images/uttower1.jpg", "images/uttower2.jpg", "SIFT", "results/uttower_stitching_sift.png")
stitch_images("images/uttower1.jpg", "images/uttower2.jpg", "HOG", "results/uttower_stitching_hog.png")

In [75]:
# 拼接多张图像
stitch_multiple_images(["images/yosemite1.jpg", "images/yosemite2.jpg", "images/yosemite3.jpg", "images/yosemite4.jpg"], "results/yosemite_stitching.png")